In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
import scipy
from sklearn import preprocessing

2024-02-14 17:58:57.346423: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-14 17:58:58.285347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 17:58:58.285369: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 17:58:58.288207: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 17:58:58.865504: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-14 17:58:58.867064: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
attacks = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_processed.csv", sep=",")

In [3]:
attacks = preprocessing.normalize(attacks.to_numpy(), norm="max", axis=0)

# swap sport to front
attacks[:, [0, 1]] = attacks[:, [1, 0]]
# swap dport to after sport
attacks[:, [1, 2]] = attacks[:, [2, 1]]
# swap protocols to after dport
attacks[:, [2, 14]] = attacks[:, [14, 2]]

# column order now -> sport, dport, protocols, continuous (discrete, discrete, discrete, continuous)

# should probably add batch and shuffle

train_dataset = attacks[:int(np.floor(attacks.shape[0]*3/4))]
test_dataset = attacks[int(np.floor(attacks.shape[0]*3/4)):]

num_features = attacks[:int(np.floor(attacks.shape[0]*3/4))].shape[1]
seq_length = 5
latent_dim = 100

In [4]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return tf.keras.backend.random_normal(tf.shape(log_var)) * tf.keras.backend.exp(log_var / 2) + mean

In [5]:
# encoder

inputs = tf.keras.layers.Input(shape=[num_features,1])
z = tf.keras.layers.Flatten()(inputs)
z = tf.keras.layers.Dense(150, activation="relu", kernel_initializer="he_normal")(z)
z = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(z)
codings_mean = tf.keras.layers.Dense(latent_dim)(z) # mu
codings_log_var = tf.keras.layers.Dense(latent_dim)(z) # gamma
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = tf.keras.Model(inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

# decoder

decoder_inputs = tf.keras.layers.Input(shape=[latent_dim])
x = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(decoder_inputs)
x = tf.keras.layers.Dense(150, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(num_features*1, activation="sigmoid")(x)
outputs = tf.keras.layers.Reshape([num_features, 1])(x)
variational_decoder = tf.keras.Model(inputs=[decoder_inputs], outputs=[outputs])

# VAE model

_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = tf.keras.Model(inputs=[inputs], outputs=[reconstructions])

# VAE losses

latent_loss = -0.5 * tf.keras.backend.sum(1 + codings_log_var - tf.keras.backend.exp(codings_log_var) - tf.keras.backend.square(codings_mean), axis=-1)
variational_ae.add_loss(tf.keras.backend.mean(latent_loss / float(num_features)))
# variational_ae.add_loss(tf.keras.backend.mean(latent_loss))
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-5)
variational_ae.compile(loss="binary_crossentropy", optimizer=optimizer)

In [6]:
def wasserstein_loss(y_pred, y_real):
    return tf.keras.backend.mean(y_pred * y_real)

def generator_wasserstein_loss(y_pred):
    return -tf.keras.backend.mean(y_pred)

class ClipConstraint(tf.keras.constraints.Constraint):
    def __init__(self, clip_value):
        self.clip_value = clip_value

In [51]:
def make_critic(input_shape):
    init = tf.keras.initializers.RandomNormal(stddev=0.02)
    const = ClipConstraint(0.01)
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.Flatten(input_shape=(input_shape[0]*input_shape[1])))
    model.add(tf.keras.layers.LSTM(100, input_shape=input_shape, kernel_initializer=init, kernel_constraint=const, recurrent_dropout=0.4, return_sequences=True, name="c_lstm1"))
    model.add(tf.keras.layers.LSTM(100, kernel_initializer=init, kernel_constraint=const, recurrent_dropout=0.4, return_sequences=True, name="c_lstm2"))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.LSTM(100, kernel_initializer=init, kernel_constraint=const, recurrent_dropout=0.4, name="c_lstm3"))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Dense(1, name="c_dense"))
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
    model.compile(loss=wasserstein_loss, optimizer=opt)
    return model

def make_generator(output_shape):
    init = tf.keras.initializers.RandomNormal(stddev=0.02)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(100, input_shape=(5, output_shape), kernel_initializer=init, recurrent_dropout=0.4, return_sequences=True))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.LSTM(100, kernel_initializer=init, recurrent_dropout=0.4))
    model.add(tf.keras.layers.ReLU())
    # model.add(tf.keras.layers.Dense(output_shape[0]*output_shape[1], activation="tanh", kernel_initializer=init))
    # model.add(tf.keras.layers.Reshape([output_shape[0], output_shape[1]]))
    model.add(tf.keras.layers.Dense(output_shape))
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
    model.compile(loss=generator_wasserstein_loss, optimizer=opt)
    return model

def make_gan(generator, critic):
    model = tf.keras.models.Sequential()
    model.add(generator)
    model.add(critic)
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
    model.compile(loss=wasserstein_loss, optimizer=opt)
    return model

## Step 1: Train autoencoder on real data

In [14]:
variational_ae.fit(train_dataset, train_dataset, epochs=2, batch_size=64)

Epoch 1/2
31151/31151 [==============================] - 30s 951us/step - loss: 0.2165
Epoch 2/2
31151/31151 [==============================] - 29s 946us/step - loss: 0.2147


## Step 2: Train GAN to learn realistic latent vectors

In [15]:
ix = np.random.randint(0, test_dataset.shape[0], 64)
real_latent_vectors = variational_encoder(test_dataset[ix])

In [59]:

# train the generator and critic
def train(generator, critic, gan, dataset, latent_dim, n_epochs=2, n_batch=64, n_critic=5):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# lists for keeping track of loss
	c1_hist, c2_hist, g_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# update the critic more than the generator
		c1_tmp, c2_tmp = list(), list()
		for _ in range(n_critic):
			# get randomly selected 'real' samples
			ix = np.random.randint(0, dataset.shape[0], n_batch)
			real_latent_vectors = variational_encoder(dataset[ix])[0]
			y_real = -np.ones((n_batch, 1))
			# update critic model weights
			c_loss1 = critic.train_on_batch(real_latent_vectors, y_real)
			c1_tmp.append(c_loss1)
			# generate 'fake' examples
			# noise = np.random.randn(latent_dim * n_batch)
			# noise = noise.reshape(32, n_batch, latent_dim)
			noise = tf.random.normal((n_batch, 5, latent_dim))
			# fake_latent_vectors = generator.predict(noise)
			fake_latent_vectors = generator(noise)
			y_fake = np.ones((n_batch, 1))
			# update critic model weights
			c_loss2 = critic.train_on_batch(fake_latent_vectors, y_fake)
			c2_tmp.append(c_loss2)
		# store critic loss
		c1_hist.append(np.mean(c1_tmp))
		c2_hist.append(np.mean(c2_tmp))
		# prepare points in latent space as input for the generator
		# noise = np.random.randn(latent_dim * n_batch)
		# noise = noise.reshape(n_batch, latent_dim)
		noise = tf.random.normal((n_batch, 5, latent_dim))
		# create inverted labels for the fake samples
		y_gan = -np.ones((n_batch, 1))
		# update the generator via the critic's error
		g_loss = gan.train_on_batch(noise, y_gan)
		g_hist.append(g_loss)
		# summarize loss on this batch
		print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))

In [53]:
real_latent_vectors = variational_encoder(test_dataset)[0]
real_latent_vectors.shape

TensorShape([664542, 100])

In [60]:
# size of the latent space
latent_dim = 100
n_batch = 64
# create the critic
critic = make_critic([latent_dim, 1])
# create the generator
# generator = make_generator((n_batch, latent_dim))
generator = make_generator(latent_dim)
# create the gan
gan = make_gan(generator, critic)
# load image data


real_latent_vectors = variational_encoder(test_dataset)[0]
epochs = 2
# train model
train(generator, critic, gan, dataset=test_dataset, latent_dim=latent_dim, n_epochs=epochs, n_batch=n_batch)

>1, c1=-0.000, c2=0.001 g=-0.000
>2, c1=-0.002, c2=0.002 g=-0.002
>3, c1=-0.003, c2=0.003 g=-0.003
>4, c1=-0.004, c2=0.004 g=-0.004
>5, c1=-0.004, c2=0.005 g=-0.004
>6, c1=-0.005, c2=0.006 g=-0.005
>7, c1=-0.006, c2=0.006 g=-0.006
>8, c1=-0.006, c2=0.007 g=-0.006
>9, c1=-0.007, c2=0.007 g=-0.007
>10, c1=-0.008, c2=0.008 g=-0.008
>11, c1=-0.008, c2=0.009 g=-0.008
>12, c1=-0.009, c2=0.009 g=-0.009
>13, c1=-0.009, c2=0.010 g=-0.009
>14, c1=-0.010, c2=0.010 g=-0.010
>15, c1=-0.010, c2=0.011 g=-0.010
>16, c1=-0.011, c2=0.011 g=-0.011
>17, c1=-0.012, c2=0.012 g=-0.011
>18, c1=-0.012, c2=0.013 g=-0.012
>19, c1=-0.013, c2=0.013 g=-0.013
>20, c1=-0.013, c2=0.014 g=-0.013
>21, c1=-0.014, c2=0.014 g=-0.014
>22, c1=-0.014, c2=0.015 g=-0.014
>23, c1=-0.015, c2=0.015 g=-0.015
>24, c1=-0.016, c2=0.016 g=-0.015
>25, c1=-0.016, c2=0.017 g=-0.016
>26, c1=-0.017, c2=0.017 g=-0.017
>27, c1=-0.017, c2=0.018 g=-0.017
>28, c1=-0.018, c2=0.018 g=-0.018
>29, c1=-0.018, c2=0.019 g=-0.018
>30, c1=-0.019, c2=0.02

KeyboardInterrupt: 

## Step 3: Use generator to make latent vectors and decode with decoder

In [62]:
noise = tf.random.normal((n_batch, 5, latent_dim))
latent_vectors = generator.predict(noise)
data = variational_decoder(latent_vectors)
data[0]

2/2 [==============================] - 0s 2ms/step


<tf.Tensor: shape=(23, 1), dtype=float32, numpy=
array([[0.36855614],
       [0.44758886],
       [0.45823923],
       [0.46371692],
       [0.26560047],
       [0.28201348],
       [0.28961593],
       [0.22479907],
       [0.35500348],
       [0.18665223],
       [0.28441554],
       [0.35116264],
       [0.259017  ],
       [0.25053042],
       [0.46333748],
       [0.3667369 ],
       [0.37243116],
       [0.2580921 ],
       [0.3599453 ],
       [0.36139977],
       [0.36782458],
       [0.23631959],
       [0.36715466]], dtype=float32)>